# Using Zero-Shot Classification for Sentiment Analysis

## 1. Load in your libraries

In [ ]:
# load hugging face library and model


from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli")

# load in pandas and numpy for data manipulation
import pandas as pd
import numpy as np

## 2. Read in your data

In [73]:
# reading in data 
df = pd.read_csv("https://raw.githubusercontent.com/akaba09/redditmentalhealth/main/files/dep.csv")

In [11]:
df.head()

,subreddit,date,text,covid period,gender
0,depression,2019-08-28,the sun hasnt even came up and im f ready to g...,pre,female
1,depression,2019-08-28,i am just so exhausted im f currently in an in...,pre,female
2,depression,2019-10-08,not sure if i am depressed burnt out or if soc...,pre,female
3,depression,2019-10-08,hi this is my first post just wanted to talk t...,pre,female
4,depression,2018-11-07,i f felt like i had my shit together at now i ...,pre,female


## 3.  Create a list of classes that you want to use for predicting sentiment

In [74]:
# Creating a list of emotions to use as labels
text_labels = ["anticipation", "anger", "disgust", "fear", "joy", "trust"]

## 4. Run the model prediction on one piece of text first


In [75]:
# example text
sample_text = " still have depression symptoms not as bad as they used to be in fact my therapist says im improving a lot but for the past years ive been stuck in this state of emotional numbness feeling disconnected from myself others and the world and time doesnt seem to be passing"


In [20]:
# run prediction
classifier(sample_text, text_labels, multi_label = False)

{'sequence': ' still have depression symptoms not as bad as they used to be in fact my therapist says im improving a lot but for the past years ive been stuck in this state of emotional numbness feeling disconnected from myself others and the world and time doesnt seem to be passing',
 'labels': ['anticipation', 'trust', 'joy', 'disgust', 'fear', 'anger'],
 'scores': [0.6039842963218689,
  0.1163715273141861,
  0.074860118329525,
  0.07247171550989151,
  0.0699692890048027,
  0.0623430535197258]}

## 5. Write a custom function to make predictions on the entire dataset, and include the labels as part of the dataframe

In [67]:
def predict_sentiment(df, text_column, text_labels):
    """
    Predict the sentiment for a piece of text in a dataframe.

    Args:
        df (pandas.DataFrame): A DataFrame containing the text data to perform sentiment analysis on.
        text_column (str): The name of the column in the DataFrame that contains the text data.
        text_labels (list): A list of text labels for sentiment classification.

    Returns:
        pandas.DataFrame: A DataFrame containing the original data with additional columns for the predicted 
        sentiment label and corresponding score.

    Raises:
        ValueError: If the DataFrame (df) does not contain the specified text_column.

    Example:
        # Assuming df is a pandas DataFrame and text_labels is a list of text labels
        result = predict_sentiment(df, "text_column_name", text_labels)
    """
    
    result_list = []
    for index, row in df.iterrows():
        sequence_to_classify = row[text_column]
        result = classifier(sequence_to_classify, text_labels, multi_label = False)
        result['sentiment'] = result['labels'][0]
        result['score'] = result['scores'][0]
        result_list.append(result)
    result_df = pd.DataFrame(result_list)[['sequence', 'sentiment', 'score']]
    result_df = pd.merge(df, result_df, left_on = "text", right_on="sequence", how = "left")
    return result_df

In [68]:
# run prediction on df
results_df = predict_sentiment(df=df, text_column ="text", text_labels= text_labels)

In [71]:
results_df.head()

,subreddit,date,text,covid period,gender,sequence,sentiment,score
0,depression,2019-08-28,the sun hasnt even came up and im f ready to g...,pre,female,the sun hasnt even came up and im f ready to g...,anticipation,0.858212
1,depression,2019-08-28,i am just so exhausted im f currently in an in...,pre,female,i am just so exhausted im f currently in an in...,anticipation,0.699415
2,depression,2019-10-08,not sure if i am depressed burnt out or if soc...,pre,female,not sure if i am depressed burnt out or if soc...,anticipation,0.472029
3,depression,2019-10-08,hi this is my first post just wanted to talk t...,pre,female,hi this is my first post just wanted to talk t...,anticipation,0.866449
4,depression,2018-11-07,i f felt like i had my shit together at now i ...,pre,female,i f felt like i had my shit together at now i ...,anticipation,0.472515
